# Isolation Forest Model Training for Digital Threat Fingerprint

This notebook trains an Isolation Forest model for anomaly detection based on behavioral features.

## Features:
- total_events: Total number of events in the 10-minute window
- update_mobile_attempt_count: Count of update_mobile_attempt events
- events_per_minute: Average events per minute

## Model Configuration:
- Algorithm: Isolation Forest
- Contamination: 0.1 (10% of data expected to be anomalies)
- Output: Model saved to `ml/models/isoforest_absher.pkl`


In [1]:
# Import required libraries
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pickle
import os
import joblib
from datetime import datetime, timedelta


## Step 1: Generate or Load Training Data

For this hackathon/PoC, we'll generate synthetic training data that simulates normal and anomalous behavior patterns similar to Tawakkalna/Absher applications.


In [2]:
# Generate synthetic training data
# IMPORTANT: Isolation Forest should be trained on NORMAL data only
np.random.seed(42)

print("=" * 60)
print("Generating Training Data")
print("=" * 60)

# ===== NORMAL BEHAVIOR PATTERNS (1000 sessions) =====
# Normal users: low event rates, occasional service access
print("\n1. Generating 1000 normal sessions...")
normal_events_total = np.random.poisson(5, 1000)  # 0-15 events per 10 min
normal_update_attempts = np.random.poisson(0.5, 1000)  # Very few update attempts (0-2 typically)
normal_events_per_min = normal_events_total / 10.0  # Normal rate: ~0.5 events/min

# Ensure minimum values
normal_events_total = np.maximum(normal_events_total, 1)  # At least 1 event
normal_events_per_min = np.maximum(normal_events_per_min, 0.1)  # At least 0.1 events/min

# Combine normal features
X_train = np.column_stack([
    normal_events_total,
    normal_update_attempts,
    normal_events_per_min
])

print(f"   Normal data shape: {X_train.shape}")
print(f"   Normal events range: [{normal_events_total.min()}, {normal_events_total.max()}]")
print(f"   Normal update attempts range: [{normal_update_attempts.min()}, {normal_update_attempts.max()}]")
print(f"   Normal events/min range: [{normal_events_per_min.min():.2f}, {normal_events_per_min.max():.2f}]")

# ===== ANOMALOUS BEHAVIOR PATTERNS (50 attack sessions) =====
# High-speed attacks: many events in short time
print("\n2. Generating 50 attack sessions for testing...")
anomaly_events_total = np.random.poisson(25, 50)  # 15-40 events per 10 min
anomaly_update_attempts = np.random.poisson(8, 50)  # Many update attempts (5-15, suspicious)
# For attacks, events happen in compressed time (e.g., 2 minutes instead of 10)
anomaly_events_per_min = anomaly_events_total / 2.0  # Very high rate: 5-20 events/min

# Ensure minimum values for attacks
anomaly_events_total = np.maximum(anomaly_events_total, 15)  # At least 15 events
anomaly_update_attempts = np.maximum(anomaly_update_attempts, 5)  # At least 5 update attempts
anomaly_events_per_min = np.maximum(anomaly_events_per_min, 5.0)  # At least 5 events/min

# Combine anomaly features (for testing only, NOT for training)
X_test_anomaly = np.column_stack([
    anomaly_events_total,
    anomaly_update_attempts,
    anomaly_events_per_min
])

print(f"   Attack data shape: {X_test_anomaly.shape}")
print(f"   Attack events range: [{anomaly_events_total.min()}, {anomaly_events_total.max()}]")
print(f"   Attack update attempts range: [{anomaly_update_attempts.min()}, {anomaly_update_attempts.max()}]")
print(f"   Attack events/min range: [{anomaly_events_per_min.min():.2f}, {anomaly_events_per_min.max():.2f}]")

# Create labels for evaluation (0 = normal, 1 = anomaly)
y_test = np.hstack([np.zeros(1000), np.ones(50)])

print("\n" + "=" * 60)
print("Summary:")
print(f"  Training data (normal only): {X_train.shape[0]} samples")
print(f"  Test data (anomaly): {X_test_anomaly.shape[0]} samples")
print(f"  Features: {X_train.shape[1]} (total_events, update_mobile_attempt_count, events_per_minute)")
print("=" * 60)


Generating Training Data

1. Generating 1000 normal sessions...
   Normal data shape: (1000, 3)
   Normal events range: [1, 12]
   Normal update attempts range: [0, 3]
   Normal events/min range: [0.10, 1.20]

2. Generating 50 attack sessions for testing...
   Attack data shape: (50, 3)
   Attack events range: [17, 36]
   Attack update attempts range: [5, 15]
   Attack events/min range: [8.50, 18.00]

Summary:
  Training data (normal only): 1000 samples
  Test data (anomaly): 50 samples
  Features: 3 (total_events, update_mobile_attempt_count, events_per_minute)


## Step 2: Feature Scaling (Optional but Recommended)

Isolation Forest is generally robust to scale, but scaling can help with consistency.


In [3]:
# Optional: Scale features
# Note: Isolation Forest works well without scaling, but scaling can improve consistency
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_train)

print("Feature statistics (original):")
print(f"Mean: {X_train.mean(axis=0)}")
print(f"Std: {X_train.std(axis=0)}")

print("\nFeature statistics (scaled):")
print(f"Mean: {X_scaled.mean(axis=0)}")
print(f"Std: {X_scaled.std(axis=0)}")


Feature statistics (original):
Mean: [4.99  0.469 0.499]
Std: [2.2252865  0.6906801  0.22252865]

Feature statistics (scaled):
Mean: [-6.59472477e-17  1.12576615e-16 -5.77315973e-18]
Std: [1. 1. 1.]


## Step 3: Train Isolation Forest Model

Configure the Isolation Forest with appropriate parameters:
- `contamination`: Expected proportion of anomalies (0.1 = 10%)
- `n_estimators`: Number of trees (more = better but slower)
- `random_state`: For reproducibility


In [4]:
# Train Isolation Forest model
# CRITICAL: Train on NORMAL data only (X_train)
# Isolation Forest learns what "normal" looks like, then flags anything different

print("=" * 60)
print("Training Isolation Forest Model")
print("=" * 60)
print("\n⚠️  IMPORTANT: Training on NORMAL data only!")
print(f"   Training samples: {X_train.shape[0]}")

iso_forest = IsolationForest(
    contamination=0.1,  # Expect 10% of data to be anomalies (used for threshold)
    n_estimators=100,   # Number of trees (more = better but slower)
    max_samples='auto', # Sample size for each tree
    random_state=42,    # For reproducibility
    n_jobs=-1          # Use all CPU cores
)

# Fit the model on NORMAL data only
print("\nTraining model...")
iso_forest.fit(X_train)

print("\n✅ Model trained successfully!")
print(f"   Number of estimators: {iso_forest.n_estimators}")
print(f"   Contamination parameter: {iso_forest.contamination}")
print(f"   Max samples per tree: {iso_forest.max_samples}")


Training Isolation Forest Model

⚠️  IMPORTANT: Training on NORMAL data only!
   Training samples: 1000

Training model...

✅ Model trained successfully!
   Number of estimators: 100
   Contamination parameter: 0.1
   Max samples per tree: auto


## Step 4: Evaluate Model Performance

Check how well the model identifies anomalies in the training data.


In [5]:
# Evaluate model performance
# Test on both normal training data and anomaly test data
from sklearn.metrics import classification_report, confusion_matrix

print("=" * 60)
print("Model Evaluation")
print("=" * 60)

# Combine test data
X_test_all = np.vstack([X_train, X_test_anomaly])

# Get predictions (-1 = anomaly, 1 = normal)
predictions = iso_forest.predict(X_test_all)

# Get decision scores (lower = more anomalous)
decision_scores = iso_forest.decision_function(X_test_all)

# Separate scores for normal and anomaly
normal_scores = decision_scores[:len(X_train)]
anomaly_scores = decision_scores[len(X_train):]

print(f"\n📊 Decision Score Statistics:")
print(f"   Normal data scores:")
print(f"     Mean: {normal_scores.mean():.3f}")
print(f"     Min: {normal_scores.min():.3f}")
print(f"     Max: {normal_scores.max():.3f}")
print(f"     Std: {normal_scores.std():.3f}")

print(f"\n   Attack data scores:")
print(f"     Mean: {anomaly_scores.mean():.3f}")
print(f"     Min: {anomaly_scores.min():.3f}")
print(f"     Max: {anomaly_scores.max():.3f}")
print(f"     Std: {anomaly_scores.std():.3f}")

# Check if anomaly scores are clearly negative
print(f"\n✅ Validation:")
if anomaly_scores.mean() < -0.15:
    print(f"   ✓ Attack scores are clearly negative (mean: {anomaly_scores.mean():.3f})")
    print(f"   ✓ Model should detect attacks effectively")
else:
    print(f"   ⚠️  Warning: Attack scores may not be negative enough")
    print(f"   ⚠️  Mean attack score: {anomaly_scores.mean():.3f} (should be < -0.15)")

# Convert to binary labels for evaluation
predicted_labels = (predictions == -1).astype(int)

print(f"\n📈 Classification Performance:")
print("\nConfusion Matrix:")
cm = confusion_matrix(y_test, predicted_labels)
print(cm)
print(f"   True Negatives (Normal detected as Normal): {cm[0,0]}")
print(f"   False Positives (Normal detected as Attack): {cm[0,1]}")
print(f"   False Negatives (Attack detected as Normal): {cm[1,0]}")
print(f"   True Positives (Attack detected as Attack): {cm[1,1]}")

print("\nClassification Report:")
print(classification_report(y_test, predicted_labels, target_names=['Normal', 'Anomaly']))

print(f"\n📉 Decision Score Range:")
print(f"   Overall range: [{decision_scores.min():.3f}, {decision_scores.max():.3f}]")
print(f"   Normal range: [{normal_scores.min():.3f}, {normal_scores.max():.3f}]")
print(f"   Anomaly range: [{anomaly_scores.min():.3f}, {anomaly_scores.max():.3f}]")

# Store reference values for engine.py
MAX_NORMAL_SCORE = max(0.1, normal_scores.max())
MIN_ANOMALY_SCORE = min(-0.2, anomaly_scores.min())

print(f"\n💡 Reference Values for engine.py:")
print(f"   MAX_NORMAL_SCORE = {MAX_NORMAL_SCORE:.3f}")
print(f"   MIN_ANOMALY_SCORE = {MIN_ANOMALY_SCORE:.3f}")
print(f"\n   These values will be used in the risk score conversion formula.")


Model Evaluation

📊 Decision Score Statistics:
   Normal data scores:
     Mean: 0.105
     Min: -0.137
     Max: 0.169
     Std: 0.068

   Attack data scores:
     Mean: -0.184
     Min: -0.184
     Max: -0.184
     Std: 0.000

✅ Validation:
   ✓ Attack scores are clearly negative (mean: -0.184)
   ✓ Model should detect attacks effectively

📈 Classification Performance:

Confusion Matrix:
[[915  85]
 [  0  50]]
   True Negatives (Normal detected as Normal): 915
   False Positives (Normal detected as Attack): 85
   False Negatives (Attack detected as Normal): 0
   True Positives (Attack detected as Attack): 50

Classification Report:
              precision    recall  f1-score   support

      Normal       1.00      0.92      0.96      1000
     Anomaly       0.37      1.00      0.54        50

    accuracy                           0.92      1050
   macro avg       0.69      0.96      0.75      1050
weighted avg       0.97      0.92      0.94      1050


📉 Decision Score Range:
   Ove

## Step 5: Test on Example Cases

Test the model on specific example scenarios.


In [6]:
# Test on specific example scenarios
print("=" * 60)
print("Testing on Example Scenarios")
print("=" * 60)

# Example 1: Normal user behavior
normal_example = np.array([[3, 0, 0.3]])  # 3 events, 0 update attempts, 0.3 events/min
pred_normal = iso_forest.predict(normal_example)[0]
score_normal = iso_forest.decision_function(normal_example)[0]

print(f"\n📊 Example 1: Normal User Behavior")
print(f"   Features: [total_events={normal_example[0,0]}, update_attempts={normal_example[0,1]}, events_per_min={normal_example[0,2]:.2f}]")
print(f"   Prediction: {'🚨 Anomaly' if pred_normal == -1 else '✅ Normal'}")
print(f"   Decision score: {score_normal:.3f}")
print(f"   Expected: Positive score (normal behavior)")

# Example 2: High-speed attack (20 events in 2 minutes)
attack_example = np.array([[20, 15, 10.0]])  # 20 events, 15 update attempts, 10 events/min
pred_attack = iso_forest.predict(attack_example)[0]
score_attack = iso_forest.decision_function(attack_example)[0]

print(f"\n📊 Example 2: High-Speed Attack")
print(f"   Features: [total_events={attack_example[0,0]}, update_attempts={attack_example[0,1]}, events_per_min={attack_example[0,2]:.2f}]")
print(f"   Prediction: {'🚨 Anomaly' if pred_attack == -1 else '✅ Normal'}")
print(f"   Decision score: {score_attack:.3f}")
print(f"   Expected: Negative score < -0.15 (attack behavior)")

# Test risk score conversion (as in engine.py)
MAX_NORMAL_SCORE = 0.1
MIN_ANOMALY_SCORE = -0.2

def calculate_risk_score(raw_score, max_normal=MAX_NORMAL_SCORE, min_anomaly=MIN_ANOMALY_SCORE):
    """Convert decision score to risk score (0-100)"""
    risk = 100 * (1.0 - (raw_score - min_anomaly) / (max_normal - min_anomaly))
    return max(0, min(100, int(risk)))

normal_risk = calculate_risk_score(score_normal)
attack_risk = calculate_risk_score(score_attack)

print(f"\n🎯 Risk Score Conversion:")
print(f"   Normal example risk score: {normal_risk} (should be < 80)")
print(f"   Attack example risk score: {attack_risk} (should be >= 80)")
print(f"\n   {'✅ Attack would be detected!' if attack_risk >= 80 else '⚠️  Attack may not be detected (risk < 80)'}")


Testing on Example Scenarios

📊 Example 1: Normal User Behavior
   Features: [total_events=3.0, update_attempts=0.0, events_per_min=0.30]
   Prediction: ✅ Normal
   Decision score: 0.151
   Expected: Positive score (normal behavior)

📊 Example 2: High-Speed Attack
   Features: [total_events=20.0, update_attempts=15.0, events_per_min=10.00]
   Prediction: 🚨 Anomaly
   Decision score: -0.184
   Expected: Negative score < -0.15 (attack behavior)

🎯 Risk Score Conversion:
   Normal example risk score: 0 (should be < 80)
   Attack example risk score: 94 (should be >= 80)

   ✅ Attack would be detected!


## Step 6: Save the Trained Model

Save the model to `ml/models/isoforest_absher.pkl` for use in the threat engine.


In [ ]:
# Get the absolute path to save the model
# Method 1: Try to get the notebook's directory
import sys
import pathlib

# Get current working directory
current_dir = os.getcwd()
print(f"Current working directory: {current_dir}")

# Method 2: Try to find project root by looking for specific files/directories
project_root = None
search_paths = [
    current_dir,  # Start from current directory
    os.path.dirname(current_dir),  # Parent directory
    os.path.dirname(os.path.dirname(current_dir)),  # Grandparent
]

# Look for project root by checking for key directories/files
for path in search_paths:
    # Check if this looks like the project root
    if (os.path.exists(os.path.join(path, 'ml', 'notebooks')) and 
        os.path.exists(os.path.join(path, 'backend')) and
        os.path.exists(os.path.join(path, 'frontend'))):
        project_root = path
        break

# If not found, try to navigate up from current directory
if project_root is None:
    test_path = current_dir
    for _ in range(5):  # Try up to 5 levels up
        if os.path.exists(os.path.join(test_path, 'ml', 'notebooks')):
            project_root = test_path
            break
        parent = os.path.dirname(test_path)
        if parent == test_path:  # Reached root
            break
        test_path = parent

# Final fallback: use relative path from notebook location
if project_root is None:
    # Assume notebook is in ml/notebooks, so go up 2 levels
    if 'notebooks' in current_dir:
        project_root = os.path.dirname(os.path.dirname(current_dir))
    else:
        # Last resort: use current directory and create ml/models there
        project_root = current_dir

print(f"Project root found: {project_root}")

# Create models directory if it doesn't exist
model_dir = os.path.join(project_root, 'ml', 'models')
model_dir = os.path.abspath(model_dir)  # Ensure absolute path
os.makedirs(model_dir, exist_ok=True)

print(f"Model directory: {model_dir}")
print(f"Directory exists: {os.path.exists(model_dir)}")

# Save the model
model_path = os.path.join(model_dir, 'isoforest_absher.pkl')
model_path = os.path.abspath(model_path)  # Ensure absolute path

print(f"\nSaving model to: {model_path}")

try:
    with open(model_path, 'wb') as f:
        pickle.dump(iso_forest, f)
    
    # Verify file was created
    if os.path.exists(model_path):
        file_size = os.path.getsize(model_path) / 1024
        print(f"\n✅ Model saved successfully!")
        print(f"   Path: {model_path}")
        print(f"   File size: {file_size:.2f} KB")
        print(f"   File exists: {os.path.exists(model_path)}")
    else:
        print("\n❌ ERROR: Model file was not created!")
        print(f"   Attempted path: {model_path}")
        
except Exception as e:
    print(f"\n❌ ERROR saving model: {e}")
    print(f"   Error type: {type(e).__name__}")
    import traceback
    traceback.print_exc()
    raise

# Verify the model can be loaded
print("\nVerifying model can be loaded...")
try:
    with open(model_path, 'rb') as f:
        loaded_model = pickle.load(f)
    
    print("✅ Model verification: Loaded successfully!")
    print(f"   Model type: {type(loaded_model)}")
    
    # Test loaded model
    test_sample = np.array([[10, 5, 1.0]])
    test_pred = loaded_model.predict(test_sample)[0]
    print(f"   Test prediction: {'Anomaly' if test_pred == -1 else 'Normal'}")
    
    # Check if we're in Colab and provide download option
    try:
        import google.colab  # type: ignore[import]  # pyright: ignore[reportMissingImports]
        print(f"\n💡 ملاحظة: أنت تعمل في Google Colab")
        print(f"   استخدم الخلية التالية لتحميل النموذج إلى جهازك")
    except ImportError:
        pass
    
except Exception as e:
    print(f"❌ ERROR loading model: {e}")
    import traceback
    traceback.print_exc()
    raise


Current working directory: /content
Project root found: /content
Model directory: /content/ml/models
Directory exists: True

Saving model to: /content/ml/models/isoforest_absher.pkl

✅ Model saved successfully!
   Path: /content/ml/models/isoforest_absher.pkl
   File size: 560.90 KB
   File exists: True

Verifying model can be loaded...
✅ Model verification: Loaded successfully!
   Model type: <class 'sklearn.ensemble._iforest.IsolationForest'>
   Test prediction: Anomaly

💡 ملاحظة: أنت تعمل في Google Colab
   استخدم الخلية التالية لتحميل النموذج إلى جهازك


## حل بديل: حفظ النموذج باستخدام مسار مباشر

إذا لم يعمل الحفظ أعلاه، استخدم هذه الخلية لحفظ النموذج مباشرة.


In [8]:
# حل بديل: حفظ النموذج مباشرة
# استخدم هذا الكود إذا لم يعمل الحفظ في الخلية السابقة

import os

# حدد المسار مباشرة (عدّل هذا المسار حسب موقع مشروعك)
# استبدل هذا بالمسار الكامل لمجلد المشروع
PROJECT_ROOT = r"c:\Users\waaad\OneDrive - ek.com.sa\Documents\hakathoon"

# أو استخدم المسار النسبي من المجلد الحالي
# PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))

# إنشاء مجلد النماذج
model_dir = os.path.join(PROJECT_ROOT, 'ml', 'models')
os.makedirs(model_dir, exist_ok=True)

# مسار الملف
model_path = os.path.join(model_dir, 'isoforest_absher.pkl')
model_path = os.path.abspath(model_path)

print(f"المسار الكامل: {model_path}")
print(f"المجلد موجود: {os.path.exists(model_dir)}")

# حفظ النموذج
try:
    with open(model_path, 'wb') as f:
        pickle.dump(iso_forest, f)
    
    if os.path.exists(model_path):
        size_kb = os.path.getsize(model_path) / 1024
        print(f"\n✅ تم حفظ النموذج بنجاح!")
        print(f"   المسار: {model_path}")
        print(f"   الحجم: {size_kb:.2f} KB")
    else:
        print("\n❌ فشل الحفظ!")
except Exception as e:
    print(f"\n❌ خطأ في الحفظ: {e}")
    print(f"\nتأكد من تعديل PROJECT_ROOT في بداية الكود")


المسار الكامل: /content/c:\Users\waaad\OneDrive - ek.com.sa\Documents\hakathoon/ml/models/isoforest_absher.pkl
المجلد موجود: True

✅ تم حفظ النموذج بنجاح!
   المسار: /content/c:\Users\waaad\OneDrive - ek.com.sa\Documents\hakathoon/ml/models/isoforest_absher.pkl
   الحجم: 560.90 KB


## تحميل النموذج من Google Colab

إذا كنت تعمل على Google Colab، استخدم هذه الخلية لتحميل النموذج المحفوظ إلى جهازك المحلي.


In [ ]:
# تحميل النموذج من Google Colab إلى جهازك المحلي
import os
import glob

# التحقق من أننا في Colab
try:
    import google.colab  # type: ignore[import]  # pyright: ignore[reportMissingImports]
    IN_COLAB = True
    print("✅ تم اكتشاف Google Colab")
except ImportError:
    IN_COLAB = False
    print("ℹ️  لا تعمل في Google Colab - لا حاجة للتحميل")

if IN_COLAB:
    from google.colab import files  # type: ignore[import]  # pyright: ignore[reportMissingImports]
    
    # البحث عن الملف في عدة مواقع محتملة
    possible_paths = [
        '/content/ml/models/isoforest_absher.pkl',
        '/content/models/isoforest_absher.pkl',
        os.path.join(os.getcwd(), 'ml', 'models', 'isoforest_absher.pkl'),
        os.path.join(os.getcwd(), 'models', 'isoforest_absher.pkl'),
    ]
    
    # البحث عن أي ملف .pkl في /content
    if not any(os.path.exists(p) for p in possible_paths):
        print("🔍 البحث عن ملف النموذج...")
        pkl_files = []
        for root, dirs, files_list in os.walk('/content'):
            for file in files_list:
                if file == 'isoforest_absher.pkl':
                    pkl_files.append(os.path.join(root, file))
        
        if pkl_files:
            possible_paths.extend(pkl_files)
            print(f"   تم العثور على {len(pkl_files)} ملف(ات)")
    
    # محاولة العثور على الملف
    model_path_colab = None
    for path in possible_paths:
        if os.path.exists(path):
            model_path_colab = path
            break
    
    if model_path_colab and os.path.exists(model_path_colab):
        print(f"📦 جاري تحميل النموذج:")
        print(f"   المسار: {model_path_colab}")
        print(f"   الحجم: {os.path.getsize(model_path_colab) / 1024:.2f} KB")
        
        try:
            # تحميل الملف
            files.download(model_path_colab)
            
            print("\n✅ تم تحميل النموذج بنجاح!")
            print("   سيبدأ التحميل تلقائياً في متصفحك")
            print(f"\n💡 بعد التحميل، انسخ الملف إلى:")
            print(f"   ml/models/isoforest_absher.pkl")
        except Exception as e:
            print(f"\n❌ خطأ في التحميل: {e}")
            print(f"\nيمكنك تحميل الملف يدوياً من:")
            print(f"   {model_path_colab}")
    else:
        print("❌ لم يتم العثور على ملف النموذج")
        print("\nتأكد من:")
        print("1. تشغيل خلية الحفظ أولاً (الخلية 13 أو 15)")
        print("2. التحقق من أن النموذج تم حفظه بنجاح")
        print(f"3. المسار الحالي: {os.getcwd()}")
        
        # عرض الملفات في مجلدات محتملة
        search_dirs = ['/content/ml/models', '/content/models', '/content']
        for search_dir in search_dirs:
            if os.path.exists(search_dir):
                print(f"\nالملفات الموجودة في {search_dir}:")
                try:
                    for f in os.listdir(search_dir):
                        full_path = os.path.join(search_dir, f)
                        if os.path.isfile(full_path):
                            size = os.path.getsize(full_path) / 1024
                            print(f"  📄 {f} ({size:.2f} KB)")
                        else:
                            print(f"  📁 {f}/")
                except Exception as e:
                    print(f"  ❌ خطأ في قراءة المجلد: {e}")
else:
    print("\n💡 إذا كنت تعمل محلياً، النموذج موجود في:")
    print("   ml/models/isoforest_absher.pkl")
    print("\nيمكنك التحقق من وجوده باستخدام:")
    print("   import os")
    print("   print(os.path.exists('ml/models/isoforest_absher.pkl'))")


✅ تم اكتشاف Google Colab
📦 جاري تحميل النموذج:
   المسار: /content/ml/models/isoforest_absher.pkl
   الحجم: 560.90 KB


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ تم تحميل النموذج بنجاح!
   سيبدأ التحميل تلقائياً في متصفحك

💡 بعد التحميل، انسخ الملف إلى:
   ml/models/isoforest_absher.pkl


## Summary

The Isolation Forest model has been trained and saved. The model will:

1. Accept 3 features: `[total_events, update_mobile_attempt_count, events_per_minute]`
2. Return prediction: `-1` for anomalies, `1` for normal
3. Provide decision scores: Lower scores indicate more anomalous behavior

### Model Usage in Engine:
The `engine.py` file loads this model and converts the decision scores to risk scores (0-100) for the Threat Fingerprint system.

### Next Steps:
- The model is ready to use in production
- Monitor performance and retrain periodically with new data
- Adjust `contamination` parameter if false positive/negative rates need tuning
